In [1]:
%config Completer.use_jedi = False
%load_ext line_profiler

import sys
sys.path.append('../')

from madina.zonal.zonal import Zonal
from madina.una.betweenness import betweenness_exposure, one_betweenness_2
import pandas as pd




## BenchmarK ettings.. 
#test_case = "Somerville" # 'NYC' , "Harvard Square", "Somerville"
#test_idx = 4


test_case = 'NYC'
test_idx = 7

# TODO: Check OS compatibility, ensure this is compatible with Unix systems..
test_case_folder = "Test Cases" + "\\" + test_case + "\\"
test_config = pd.read_csv(test_case_folder + "test_configs.csv")


print (test_config.loc[test_idx])

harvard_square = Zonal()

harvard_square.load_layer(
    layer_name='streets',
    file_path=  test_case_folder + test_config.at[test_idx, 'Network_File']
    )

if test_config.at[test_idx, 'Origin_Name'] not in harvard_square.layers.layers:
    harvard_square.load_layer(
        layer_name=test_config.at[test_idx, 'Origin_Name'],
        file_path= test_case_folder + test_config.at[test_idx, 'Origin_File']
        )

if test_config.at[test_idx, 'Destination_Name'] not in harvard_square.layers.layers:
    harvard_square.load_layer(
        layer_name=test_config.at[test_idx, 'Destination_Name'],
        file_path= test_case_folder + test_config.at[test_idx, 'Destination_File']
        )


harvard_square.create_street_network(
    source_layer='streets', 
    discard_redundant_edges=False,
    node_snapping_tolerance= 0.00001 if test_case == "NYC" else 0.01,  # TODO: check for sensitivity... pick one as default snapping.
    weight_attribute=test_config.at[test_idx, 'Network_Cost'] if test_config.at[test_idx, 'Network_Cost'] != "Geometric" else None
)


harvard_square.insert_node(
    layer_name=test_config.at[test_idx, 'Origin_Name'], 
    label='origin', 
    weight_attribute=test_config.at[test_idx, 'Origin_Weight'] if test_config.at[test_idx, 'Origin_Weight'] != "Count" else None
)
harvard_square.insert_node(
    layer_name=test_config.at[test_idx, 'Destination_Name'], 
    label='destination', 
    weight_attribute=test_config.at[test_idx, 'Destination_Weight'] if test_config.at[test_idx, 'Destination_Weight'] != "Count" else None
)

harvard_square.create_graph()

node_gdf = harvard_square.network.nodes
origin_gdf = node_gdf[node_gdf['type'] == 'origin']


harvard_square.network.turn_penalty_amount = test_config.at[test_idx, 'Turn_Penalty']
harvard_square.network.turn_threshold_degree = test_config.at[test_idx, 'Turn_Threshold']


print ("Done Preparing Benchmark")
import multiprocessing as mp



def profile_new():
    num_cores = 1
    with mp.Manager() as manager:
        #create aNDfill queue
        origin_queue = manager.Queue()
        for o_idx in origin_gdf.index[:200]:
        #for o_idx in origin_gdf.index[:1000]:
            origin_queue.put(o_idx)
        for core_index in range(num_cores):
            origin_queue.put("done")


        return_dict = betweenness_exposure(
            harvard_square,
            core_index=1,
            origin_queue=origin_queue,
            #origins=origin_gdf.iloc[:1000],
            search_radius=test_config.at[test_idx,'Radius'],
            detour_ratio=test_config.at[test_idx,'Detour'],
            decay=False if test_config.at[test_idx,'Elastic_Weights'] else test_config.at[test_idx,'Decay'],
            beta=test_config.at[test_idx,'Beta'],
            decay_method=test_config.at[test_idx,'Decay_Mode'],
            path_detour_penalty='equal',
            closest_destination=test_config.at[test_idx,'Closest_destination'],
            elastic_weight=test_config.at[test_idx,'Elastic_Weights'],
            turn_penalty=test_config.at[test_idx,'Turns'],
            path_exposure_attribute=None,
            return_path_record=False, 
            destniation_cap=None
    )
    return return_dict


def profile_baseline():
    one_betweenness_2(
        network=harvard_square.network,
        search_radius=test_config.at[test_idx,'Radius'],
        origins=origin_gdf,
        detour_ratio=test_config.at[test_idx,'Detour'],
        decay=False if test_config.at[test_idx,'Elastic_Weights'] else test_config.at[test_idx,'Decay'],
        beta=test_config.at[test_idx,'Beta'],
        decay_method=test_config.at[test_idx,'Decay_Mode'],
        path_detour_penalty="equal",  # "exponent" | "power"
        origin_weights=True,
        origin_weight_attribute=None,
        closest_destination=test_config.at[test_idx,'Closest_Destination'],
        destination_weights=True,
        destination_weight_attribute=None,
        perceived_distance=False,
        light_graph=True,
        turn_penalty=test_config.at[test_idx,'Turns'],
        retained_d_idxs=None,
        retained_paths=None,
        retained_distances=None,
        rertain_expensive_data=False
)

Unnamed: 0                                                 7
Flow_Name                  NYC_Amenities_Amenities_Geometric
Network_File           NYC_network_compiled_0726_fin.geojson
Network_Cost                                       Geometric
Origin_File                        Amenities_PT_6538.geojson
Origin_Name                                        Amenities
Origin_Weight                                          Count
Destination_File                   Amenities_PT_6538.geojson
Destination_Name                                   Amenities
Destination_Weight                                     Count
Radius                                                   800
Beta                                                   0.001
Decay                                                   True
Decay_Mode                                          exponent
Closest_destination                                    False
Detour                                                  1.15
Elastic_Weights         

# ammenities - ammennities benchmark on vectorized betweenness

In [ ]:
#%lprun  -f  profile_new profile_new()
%prun profile_new()

NYC 200 AMenities

In [ ]:
processed_origins[processed_origins['path_generation_time'] > 10]

In [2]:
import plotly.io as pio
import plotly.express as px
pio.templates.default = "plotly_dark"

import time
for chuncking_method in ['no_chunking', 'cocentric-chunks', 'random_chunks', 'pizza_chunks']:
    for chunck in [25, 50, 100, 200, 400]:
        start = time.time()
        harvard_square.network.chunking_method = chuncking_method
        harvard_square.network.max_chunck_size = chunck ## THIS IS ALREADY HARD WIRED IN THE BETWEENNESS CODE>
        profile_new()
        processed_origins = harvard_square.network.nodes[harvard_square.network.nodes['type'] == 'origin'].iloc[:1000]
        processed_origins['chunck_size'] = processed_origins['reach']  /  processed_origins['chunck_count']


        px.scatter(
            processed_origins, 
            x='path_generation_time', 
            y='reach',
            color='chunck_size',#'chunck_size', 
            title=chuncking_method if chuncking_method == 'no_chunking' else f"{chuncking_method = }\t{chunck = }"
        ).show()
        print (f"{chunck = }\t time={time.time() - start:6.2f}")
        if chuncking_method == 'no_chunking':
            break

chunck = 25	 time=2397.51


chunck = 25	 time=2361.51


chunck = 50	 time=2207.78


chunck = 100	 time=2179.18


SOmerville 1000 AMenities

In [ ]:
import time
for chunck in [25, 50, 100, 150, 200, 250, 500]:
    start = time.time()
    harvard_square.network.max_chunck_size = chunck
    profile_new()
    print (f"{chunck = }\t time={time.time() - start:6.2f}")


In [ ]:
import time 

start = time.time()
harvard_square.network.max_chunck_size = 100000
harvard_square.network.chunking_method = 'no_chunking'
profile_new()
print (f"chuncking_method =  'no_chunking'\t'hunck_size = 100000 \t time={time.time() - start:6.2f}")

for chunck_size in [250, 500]:  #[25, 50, 100, 150, 200, 250, 500]
    for chuncking_method in ['cocentric-chunks', 'random_chunks', 'pizza_chunks']:
        start = time.time()
        harvard_square.network.max_chunck_size = chunck_size
        harvard_square.network.chunking_method = chuncking_method
        profile_new()
        print (f"{chuncking_method = }\t{chunck_size = }\t time={time.time() - start:6.2f}")



In [ ]:
import resource
resource.RLIMIT_DATA

# fsst dest path algo: the wandering_messenger.
* Pretend you need to visit all destinations within search radius
* pretend you need to tell each destination, all patha that lead to it, that are longer than  shortest distance detour from origin

* pretend you are on a depth first walk. you keep track of 
*** a list of strps you took to get to this path. 
*** a queue of your depth first tree. where you append nebors, pop first neighbor. 
* Once you pop something from queue, your current path list resets to the popped neighbor's parent.
* you keep a list of visited
* at each node you check if anything in done's distance matrix is not yet visited. push (neighbor, source))

In [ ]:
import math
import numpy as np
def clockwiseangle_and_distance(origin, point):
    refvec = [0, 1]
    # Vector between point and the origin: v = p - o
    vector = [point[0]-origin[0], point[1]-origin[1]]
    # Length of vector: ||v||
    lenvector = math.hypot(vector[0], vector[1])
    # If length is zero there is no angle
    if lenvector == 0:
        return 0
    # Normalize vector: v/||v||
    normalized = [vector[0]/lenvector, vector[1]/lenvector]
    dotprod  = normalized[0]*refvec[0] + normalized[1]*refvec[1]     # x1*x2 + y1*y2
    diffprod = refvec[1]*normalized[0] - refvec[0]*normalized[1]     # x1*y2 - y1*x2
    angle = math.atan2(diffprod, dotprod)
    # Negative angles represent counter-clockwise angles so we need to subtract them 
    # from 2*pi (360 degrees)
    if angle < 0:
        return 2*math.pi+angle
    # I return first the angle because that's the primary sorting criterium
    # but if two vectors have the same angle then the shorter distance should come first.
    return angle

In [ ]:
destinations = node_gdf.loc[list(d_idxs)]
origin_geom = node_gdf.at[o_idx,'geometry'].coords
origin_point = [origin_geom[0][0], origin_geom[0][1]]
destinations['angle'] = destinations['geometry'].apply(lambda x: clockwiseangle_and_distance(origin_point, [x.coords[0][0], x.coords[0][1]]))

max_group_size = 50
for seq, dest in enumerate(destinations.sort_values('angle').index):
    destinations.at[dest, 'group'] = np.floor(seq/max_group_size) 

In [ ]:
harvard_square.create_map(
    [
        {'gdf': harvard_square.layers['streets'].gdf, 'color': [100, 100, 100], 'opacity': 1.},
        {'gdf': destinations, 'color_by_attribute': 'group', 'color_method': 'categorical', 'text':'group'},

    ]
)

In [ ]:
o_idx = 8385
harvard_square.network.add_node_to_graph(o_graph, o_idx)
d_idxs, o_scope, o_scope_paths = turn_o_scope(
    harvard_square.network,
    o_idx,
    search_radius = test_config.at[test_idx,'Radius'],
    detour_ratio = test_config.at[test_idx,'Detour'],
    turn_penalty=False,
    o_graph=o_graph,
    return_paths=False
)
harvard_square.network.remove_node_to_graph(o_graph, o_idx)

d_idxs

In [ ]:
origin_gdf.sort_values('reach', ascending=False)

# Measuring Access for each Origin

In [ ]:
from madina.una.paths import turn_o_scope

for origin_index in origin_gdf.index:
    o_graph = harvard_square.network.d_graph
    harvard_square.network.add_node_to_graph(o_graph, origin_index)
    d_idxs, o_scope, o_scope_paths = turn_o_scope(
        harvard_square.network,
        origin_index,
        search_radius = test_config.at[test_idx,'Radius'],
        detour_ratio = test_config.at[test_idx,'Detour'],
        turn_penalty=False,
        o_graph=o_graph,
        return_paths=False
    )
    harvard_square.network.remove_node_to_graph(o_graph, origin_index)
    origin_gdf.at[origin_index, 'reach'] = len(d_idxs)
    print (f"{origin_index = }\t {len(d_idxs)}")


In [ ]:
origin_gdf[origin_gdf['reach'] > 3000].explore()

In [ ]:
origin_gdf[(origin_gdf['reach'] > 2000) & (origin_gdf['reach'] <= 3000)].explore()

In [ ]:
origin_gdf[(origin_gdf['reach'] > 1000) & (origin_gdf['reach'] <= 2000)].explore()

In [ ]:
origin_gdf[(origin_gdf['reach'] > 500) & (origin_gdf['reach'] <= 1000)].explore()

In [ ]:
origin_gdf[(origin_gdf['reach'] > 250) & (origin_gdf['reach'] <= 500)].explore()

In [ ]:
origin_gdf[(origin_gdf['reach'] < 250)].explore()